In [1]:
from typing import List, Generator
import numpy
from math import comb

def generate_restrictions(upper_bounds: List[int], max_size: int, recursion_depth: int = 0, restrictions: List[int] = None) -> Generator[List[int], None, None]:
    """
    Generates all possible vectors of size n whose entries are increasing and each value is less than or equal to the
    restriction of the same position.
    Args:
        upper_bounds: A list of integers, representing the restrictions of the previous row.
        max_size: An integer, representing the dimension of the finite chain.
        recursion_depth: An integer, representing the position of the vector that is being generated.
        restrictions: A list of integers, representing the temporal vector to be generated.
    Returns:
        A generator of increasing vectors whose entries satisfy upper restrictions.
    """
    if restrictions is None:
        restrictions = []

    if recursion_depth == 0:
        for x in range(0, upper_bounds[recursion_depth]+1):
            temp_restrictions = restrictions.copy()
            temp_restrictions.append(x)
            yield from generate_restrictions(upper_bounds=upper_bounds, max_size=max_size, recursion_depth=recursion_depth + 1, restrictions=temp_restrictions)
    elif 0 < recursion_depth < max_size:
        for x in range(restrictions[recursion_depth - 1], upper_bounds[recursion_depth]+1):
            temp_restrictions = restrictions.copy()
            temp_restrictions.append(x)
            yield from generate_restrictions(upper_bounds=upper_bounds, max_size=max_size, recursion_depth=recursion_depth + 1, restrictions=temp_restrictions)
    else:
        if restrictions != upper_bounds:
            restrictions.reverse()
            yield restrictions

def counter1(n: int, restrictions: List[int]) -> float:
    mat = numpy.zeros((n, n))
    for s in range(1, n+1):
        for t in range(1, n+1):
            mat[s-1, t-1] = comb(n+restrictions[t-1], n-s+t)

    return round(numpy.linalg.det(mat))

def counter2(n: int, restrictions: List[int]) -> float:
    if is_stationary_sequence(vector=restrictions):
        return comb(n+restrictions[0]-1, restrictions[0])
    else:
        sum_value = counter1(n, restrictions)
        rev_restrictions = restrictions.copy()
        rev_restrictions.reverse()
        for vector in generate_restrictions(upper_bounds=rev_restrictions, max_size=n):
            sum_value -= counter2(n, vector)
        return sum_value

def is_stationary_sequence(vector: List[int]) -> bool:
    for i in range(1, len(vector)):
        if vector[i] != 0:
            return False
    return True

def identity(n: int) -> List[int]:
    return [i for i in range(n, 0, -1)]

def counter_np(n: int) -> int:
    return counter2(n=n, restrictions=identity(n=n))

In [2]:
for vec in generate_restrictions([0,2,3], 3):
    print(vec)

[0, 0, 0]
[1, 0, 0]
[2, 0, 0]
[3, 0, 0]
[1, 1, 0]
[2, 1, 0]
[3, 1, 0]
[2, 2, 0]


In [3]:
# IP n=3
value = 0
n = 3+1
for x1 in range(0, n):
    for x2 in range(0, n):
        for x3 in range(max(x1,x2), n):
            for x4 in range(0, n):
                for x5 in range(max(x2,x4), n):
                    value += 1
print(value)

246


In [4]:
# IP n=4
value=0
n=4+1
for x1 in range(0,n):
    for x2 in range(0,n):
        for x3 in range(max(x1,x2),n):
            for x4 in range(0,n):
                for x5 in range(max(x2,x4),n):
                    for x6 in range(max(x3,x5),n):
                        for x7 in range(0,n):
                            for x8 in range(max(x4,x7),n):
                                for x9 in range(max(x5,x8),n):
                                    value += 1
print(value)

21307


In [5]:
from math import factorial

def staircase_plane_partition(r: int, l: int, k: int, m: int) -> int:
    """
    Computes the number of staircase plane partition with r rows, step k, initial length l and maximum value m.

    Args:
        r: The number of rows of the plane partition.
        l: The initial length of the first row.
        k: The decrementing step of the length of each row.
        m: The maximum value of the plane partition.

    Returns:
        An integer, representing the number of staircase plane partitions.
    """
    x1 = 1
    for i in range(1, r+1):
        x1 *= factorial(m+l-k*i-1)/factorial(m+i-2)

    x2 = 1
    for i in range(1, r+1):
        up_fact = upper_factorial(m+(l-k*i)/(k+1), i-1)
        x2 *= up_fact/factorial(r+l-i*(k+1))

    x3 = (k+1)**comb(r, 2)

    x4 = 1
    for i in range(2, r):
        x4 *= i**(r-i)

    return round(x1*x2*x3*x4)

def upper_factorial(z: float, i: int) -> float:
    """
    Computes the upper factorial of a positive real number with respect to a positive integer.
    The upper factorial is defined as <z>_i = Prod_{j=0}^{i-1} (z-j).

    Args:
        z: A positive real number.
        i: A positive integer.

    Returns:
        The upper factorial of z with respect to i.
    """
    fact = 1
    for k in range(0, i):
        fact *= (z+k)
    return fact

In [6]:
for i in range(1, 7+1):
    n = i
    s = staircase_plane_partition(r=n, l=n+1, k=1, m=n+1)-staircase_plane_partition(r=n, l=n+1, k=1, m=n)
    print(s)

1
9
246
21307
5967884
5464753020
16464650143150


## Suavitat

In [1]:
import numpy

from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_aggregation_suboperators.conjunction import Conjunction

n = 2+1
smt = 0
smt_arch = 0

template = numpy.zeros((n,n), dtype=int)
template[2,2] = 2

for x1 in range(0, n):
    for x2 in range(x1, n):
        for x3 in range(x1, n):
            new_conj = template.copy()
            new_conj[1,1] = x1
            new_conj[1,2] = x2
            new_conj[2,1] = x3
            
            conj = Conjunction(n=2, operator_matrix=new_conj)
            if conj.is_smooth():
                smt += 1
                
                if conj.is_archimedean() and conj.is_commutative():
                    smt_arch += 1
                    print(f"Archimedean:")
                    
                    print("\t Conjunction:")
                    print(f"{new_conj}")
                    
                    print("\t Associated ASM:")
                    print(f"{conj.get_asm_representation()}")
                else:
                    print(f"Not Archimedean:")
                    
                    print("\t Conjunction:")
                    print(f"{new_conj}")
                    
                    print("\t Associated ASM:")
                    print(f"{conj.get_asm_representation()}")
                    
print(smt)
print(smt_arch)

ModuleNotFoundError: No module named 'discrete_fuzzy_operators'

In [6]:
n = 3+1
smt = 0
smt_arch = 0

template = numpy.zeros((n,n), dtype=int)
template[3,3] = 3

for x1 in range(0, n):
    for x2 in range(x1, n):
        for x3 in range(x2, n):
            for x4 in range(x1, n):
                for x5 in range(max(x2,x4), n):
                    for x6 in range(max(x3,x5), n):
                        for x7 in range(x4, n):
                            for x8 in range(max(x5,x7), n):
                                new_conj = template.copy()
                                new_conj[1,1] = x1
                                new_conj[1,2] = x2
                                new_conj[1,3] = x3
                                new_conj[2,1] = x4
                                new_conj[2,2] = x5
                                new_conj[2,3] = x6
                                new_conj[3,1] = x7
                                new_conj[3,2] = x8

                                conj = Conjunction(n=3, operator_matrix=new_conj)
                                if conj.is_smooth():
                                    smt += 1
                                    
                                    if conj.is_archimedean() and conj.is_commutative():
                                        smt_arch += 1
                                        print(f"Archimedean:")

                                        print("\t Conjunction:")
                                        print(f"{new_conj}")

                                        print("\t Associated ASM:")
                                        print(f"{conj.get_asm_representation()}")
                                    else:
                                        print(f"Not Archimedean:")

                                        print("\t Conjunction:")
                                        print(f"{new_conj}")

                                        print("\t Associated ASM:")
                                        print(f"{conj.get_asm_representation()}")
                                                      
print(smt)
print(smt_arch)

Archimedean:
	 Conjunction:
[[0 0 0 0]
 [0 0 0 1]
 [0 0 1 2]
 [0 1 2 3]]
	 Associated ASM:
[[0 0 1]
 [0 1 0]
 [1 0 0]]
Not Archimedean:
	 Conjunction:
[[0 0 0 0]
 [0 0 0 1]
 [0 1 1 2]
 [0 1 2 3]]
	 Associated ASM:
[[0 1 0]
 [0 0 1]
 [1 0 0]]
Not Archimedean:
	 Conjunction:
[[0 0 0 0]
 [0 0 1 1]
 [0 0 1 2]
 [0 1 2 3]]
	 Associated ASM:
[[0 0 1]
 [1 0 0]
 [0 1 0]]
Not Archimedean:
	 Conjunction:
[[0 0 0 0]
 [0 0 1 1]
 [0 1 1 2]
 [0 1 2 3]]
	 Associated ASM:
[[ 0  1  0]
 [ 1 -1  1]
 [ 0  1  0]]
Not Archimedean:
	 Conjunction:
[[0 0 0 0]
 [0 0 1 1]
 [0 1 2 2]
 [0 1 2 3]]
	 Associated ASM:
[[0 1 0]
 [1 0 0]
 [0 0 1]]
Not Archimedean:
	 Conjunction:
[[0 0 0 0]
 [0 1 1 1]
 [0 1 1 2]
 [0 1 2 3]]
	 Associated ASM:
[[1 0 0]
 [0 0 1]
 [0 1 0]]
Not Archimedean:
	 Conjunction:
[[0 0 0 0]
 [0 1 1 1]
 [0 1 2 2]
 [0 1 2 3]]
	 Associated ASM:
[[1 0 0]
 [0 1 0]
 [0 0 1]]
7
1


In [9]:
n = 4+1
smt = 0
smt_arch = 0

template = numpy.zeros((n,n), dtype=int)
template[4,4] = 4

for x1 in range(0, n):
    for x2 in range(x1, n):
        for x3 in range(x2, n):
            for x4 in range(x3, n):
                for x5 in range(x1, n):
                    for x6 in range(max(x2,x5), n):
                        for x7 in range(max(x3,x6), n):
                            for x8 in range(max(x4,x7), n):
                                for x9 in range(x5, n):
                                    for x10 in range(max(x6, x9), n):
                                        for x11 in range(max(x7,x10), n):
                                            for x12 in range(max(x8,x11), n):
                                                for x13 in range(x9, n):
                                                    for x14 in range(max(x10,x13), n):
                                                        for x15 in range(max(x11, x14), n):
                                                            new_conj = template.copy()
                                                            new_conj[1,1] = x1
                                                            new_conj[1,2] = x2
                                                            new_conj[1,3] = x3
                                                            new_conj[1,4] = x4
                                                            
                                                            new_conj[2,1] = x5
                                                            new_conj[2,2] = x6
                                                            new_conj[2,3] = x7
                                                            new_conj[2,4] = x8
                                                            
                                                            new_conj[3,1] = x9
                                                            new_conj[3,2] = x10
                                                            new_conj[3,3] = x11
                                                            new_conj[3,4] = x12
                                                            
                                                            new_conj[4,1] = x13
                                                            new_conj[4,2] = x14
                                                            new_conj[4,3] = x15

                                                            conj = Conjunction(n=4, operator_matrix=new_conj)
                                                            
                                                            if conj.is_smooth():
                                                                
                                                                smt += 1
                                                                
                                                                if conj.is_archimedean():
                                                                    smt_arch += 1
                                                            

                                
print(smt)
print(smt_arch)

42
7


In [10]:
from typing import Tuple
from discrete_fuzzy_operators.base.operators.binary_operators.discrete.suboperators.fuzzy_discrete_aggregation_suboperators.conjunction import Conjunction

def initial_restrictions(n: int) -> Tuple[int, int, int, int, int]:
    for x1 in range(0, 1+1):
        for x2 in range(x1, 2+1):
            for x3 in range(x2, 3+1):
                for x4 in range(x3, 4+1):
                    for x6 in range(x1, 1+1):
                        yield x1, x2, x3, x4, x6

n = 5+1
smt = 0
smt_arch = 0

template = numpy.zeros((n,n), dtype=int)
template[5,5] = 5

for x1, x2, x3, x4, x6 in initial_restrictions(n):
    for x7 in range(max(x2,x6), 2+1):
        for x8 in range(max(x3,x7), 3+1):
            for x9 in range(max(x4,x8), 4+1):
                for x11 in range(x6, 1+1):
                    for x12 in range(max(x7,x11), 2+1):
                        for x13 in range(max(x8,x12), 3+1):
                            for x14 in range(max(x9,x13), 4+1):
                                for x16 in range(x11, 1+1):
                                    for x17 in range(max(x12,x16), 2+1):
                                        for x18 in range(max(x13,x17), 3+1):
                                            for x19 in range(max(x14, x18), 4+1):
                                                
                                                new_conj = template.copy()
                                                new_conj[1,1] = x1
                                                new_conj[1,2] = x2
                                                new_conj[1,3] = x3
                                                new_conj[1,4] = x4
                                                new_conj[1,5] = 1

                                                new_conj[2,1] = x6
                                                new_conj[2,2] = x7
                                                new_conj[2,3] = x8
                                                new_conj[2,4] = x9
                                                new_conj[2,5] = 2

                                                new_conj[3,1] = x11
                                                new_conj[3,2] = x12
                                                new_conj[3,3] = x13
                                                new_conj[3,4] = x14
                                                new_conj[3,5] = 3

                                                new_conj[4,1] = x16
                                                new_conj[4,2] = x17
                                                new_conj[4,3] = x18
                                                new_conj[4,4] = x19
                                                new_conj[4,5] = 4

                                                new_conj[5,1] = 1
                                                new_conj[5,2] = 2
                                                new_conj[5,3] = 3
                                                new_conj[5,4] = 4

                                                conj = Conjunction(n=5, operator_matrix=new_conj)
                                                
                                                if conj.is_smooth():
                                                    smt += 1
                                                    
                                                    if conj.is_archimedean():
                                                        smt_arch += 1
                                                    
                                                

                                
                                
print(smt)
print(smt_arch)

429
42


In [ ]:
from math import factorial

for n in range(1, 10+1):
    value = 1
    for k in range(0, n-1+1):
        value *= factorial(3*k+1)/factorial(n+k)
    print(round(value))

## CB

In [ ]:
n = 2+1

cb_count = 0
for x1 in range(0, 1+1):
    for x2 in range(x1, n):
        for x3 in range(x1, 1+1):
            cb_count += 1
print(cb_count)

In [ ]:
n = 3+1

cb_count = 0

for x1 in range(0, 1+1):
    for x2 in range(x1, 2+1):
        for x3 in range(x2, 3+1):
            for x4 in range(x1, 1+1):
                for x5 in range(max(x2, x4), 2+1):
                    for x6 in range(max(x3, x5), 3+1):
                        for x7 in range(x4, 1+1):
                            for x8 in range(max(x5, x7), 2+1):
                                cb_count += 1
print(cb_count)

## COMMUTATIVITAT I NP

In [ ]:
count = 0
for x1 in range(0, 1+1):
    for x2 in range(x1, 1+1):
        for x3 in range(x2, 2+1):
            count += 1
print(count)

In [ ]:
count = 0
for x1 in range(0, 1+1):
    for x2 in range(x1, 1+1):
        for x3 in range(x2, 2+1):
            for x4 in range(x2, 1+1):
                for x5 in range(max(x3,x4), 2+1):
                    for x6 in range(x5, 3+1):
                        count += 1
print(count)

In [ ]:
count = 0
for x1 in range(0, 1+1):
    for x2 in range(x1, 1+1):
        for x3 in range(x2, 2+1):
            for x4 in range(x2, 1+1):
                for x5 in range(max(x3,x4), 2+1):
                    for x6 in range(x5, 3+1):
                        for x7 in range(x4, 1+1):
                            for x8 in range(max(x5, x7), 2+1):
                                for x9 in range(max(x6, x8), 3+1):
                                    for x10 in range(x9, 4+1):
                                        count += 1
print(count)